In [ ]:
# create a topic coverage matrix preset to 0
topiccoverage = []
i = 0
while i < len(docs_per_timeslice) :
    y = 0
    thistopic = []
    while y < num_topics:
        thistopic.append(0.0)
        y += 1
    topiccoverage.append(thistopic.copy())
    i += 1
# get the topic coverage per timeslice per doc
timeslice = 0
for timeslicedocs in docs_per_timeslice :
    # for each doc in this timeslice
    for doc in timeslicedocs :
        # get the probability matrix
        probs = model.get_document_topics(bow[doc]).copy()
        #its a sparse array, prob[0] is the topic and prob[1] is the probabiltiy
        for prob in probs :
            topiccoverage[timeslice][prob[0]] += prob[1]
    timeslice += 1

# ##################################################
# INPUT:
# timeslicetokencounts - we have the word coverage for each model token
# topiccoverage - and now we have the topic coverage
# topics - the current topic word probabilities
#
# OUTPUT:
# updated array newtopics = []
#
# run the iteration
# ##################################################   
newtopics = []
purity = 0.0
purity_count = 0
lowtopics = []
causality_confidence = 0.0
causality_count = 0
for ii in range(0, num_topics) :
    tempgc = grangercausalitytests([[bets[i],topiccoverage[i][ii]] for i in range(0, len(bets))], the_lag, verbose=False)

    lowpvalue = 2.0
    lowlag = 0
    for yy in range(1,the_lag+1) :
        stats = tempgc.get(yy)[0].get('ssr_ftest')
        if (stats[1] < lowpvalue) :
            lowlag = yy
            lowpvalue = stats[1]
    causality_confidence += lowpvalue
    causality_count += 1
    if lowpvalue < low_threshold :
        causality_confidence += lowpvalue
        causality_count += 1
        lowtopics.append(ii)
print ("Significant Topics", lowtopics)
for topic in lowtopics :
    pos_count = 0
    neg_count = 0
    neg_percent = 0.0
    pos_percent = 0.0
    neg_wordprob = []
    pos_wordprob = []
    wordprobstats = []
    tot = np.longdouble(0)
    count = 0

    for wordprob in topics[topic] :
        sig = float(wordprob * pgranger[count])
        tot += wordprob
        wordlist = [abs(sig), pgranger[count], count, wordprob]
        wordprobstats.append(wordlist.copy())
        count += 1
    for wordprobstat in wordprobstats :
        wordprobstat[0] = wordprobstat[0]/tot # normalize

    wordprobstats = sorted(wordprobstats, key = lambda x: float(x[0]), reverse = True)

    hit_limit = 0
    for wordprobstat in wordprobstats :
        if wordprobstat[1] > 0  :
            if hit_limit == 0 :
                pos_wordprob.append(wordprobstat.copy())
                pos_percent += wordprobstat[3]
                pos_count += 1
        if wordprobstat[1] < 0   :
            if hit_limit == 0 :
                neg_wordprob.append(wordprobstat.copy())
                neg_percent += wordprobstat[3]
                neg_count += 1
        if neg_percent + pos_percent > drop_percent or wordprobstat[0] < .0005:
            hit_limit = 1
    addpurity = 0
    if pos_count > neg_count * ignore_little_counts :
        # create new topics, start with zero prob
        createtopic = zeroprobs.copy()
        # add in these words
        #print ("OY Pos Words = ", len(pos_wordprob))
        pos_count = len(pos_wordprob)
        addpurity = 1
        for wordprob in pos_wordprob :
            createtopic[wordprob[2]] = wordprob[3]/pos_percent
            #createtopic[wordprob[2]] = 1.0
        newtopics.append(createtopic.copy())

    if neg_count > pos_percent * ignore_little_counts :
        # create new topics
        # create new topics, start with zero prob
        createtopic = zeroprobs.copy()
        # add in these words
        #print ("OY Neg Words = ", len(neg_wordprob))
        neg_count = len(neg_wordprob)
        add_purity = 1
        for wordprob in neg_wordprob :
            createtopic[wordprob[2]] = wordprob[3]/neg_percent
            #createtopic[wordprob[2]] = 1.0
        newtopics.append(createtopic.copy())
    purity_count += addpurity
    ppurity = 0
    npurity = 0
    if pos_count + neg_count != 0 :
        ppurity = pos_count / (pos_count + neg_count)
    if pos_count + neg_count != 0 :
        npurity = neg_count / (pos_count + neg_count)
    if (ppurity != 0) :
        ppurity = (ppurity * math.log(ppurity, 10))
    if (npurity != 0) :
        npurity = (npurity * math.log(npurity, 10))
    entropy =  npurity + ppurity
    #print(float(100 + (100 * entropy)))
    purity += float(100 + (100 * entropy))

#print (newtopics)
causality_confidence = 1-causality_confidence/causality_count 
print("causality_confidence = ", causality_confidence)
run_confidence.append(causality_confidence.copy())

purity = purity/purity_count
print("purity = ", purity)
run_purity.append(purity)

#%run ITMTF_iterate.ipynb

# ##################################################
#
# using the returned topics probabilies
# and adding buffers - num_buffers using bufferprob
# create the prior
# update num_topics
# and run the model
# ##################################################

# add the buffers
z = 0
while z < num_buffers:
    newtopics.append(bufferprob.copy())
    z += 1
num_topics = len(newtopics)

